In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [24]:
# Define transforms for training and testing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define directory paths (adjust if needed)
train_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/train"
val_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/val"
test_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/test"

# Create datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=test_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))
print("Test samples:", len(test_dataset))


Train samples: 5216
Validation samples: 16
Test samples: 624


In [25]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Get class names (e.g., ['NORMAL', 'PNEUMONIA'])
class_names = train_dataset.classes
print("Classes:", class_names)


Classes: ['NORMAL', 'PNEUMONIA']


In [26]:
# Create the EfficientNet-B0 model with pretrained weights
model = timm.create_model('efficientnet_b0', pretrained=True)

# For EfficientNet from timm, the final classifier layer is 'classifier'
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)

model = model.to(device)
print(model)


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [27]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        total += inputs.size(0)
    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += inputs.size(0)
    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 30
for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")



Epoch 1/30 - Train Loss: 0.1368, Train Acc: 0.9586 | Test Loss: 0.8082, Test Acc: 0.7404
Epoch 2/30 - Train Loss: 0.0363, Train Acc: 0.9887 | Test Loss: 0.4392, Test Acc: 0.8862
Epoch 3/30 - Train Loss: 0.0187, Train Acc: 0.9952 | Test Loss: 0.6130, Test Acc: 0.8542
Epoch 4/30 - Train Loss: 0.0136, Train Acc: 0.9965 | Test Loss: 0.9294, Test Acc: 0.7965
Epoch 5/30 - Train Loss: 0.0100, Train Acc: 0.9965 | Test Loss: 0.9842, Test Acc: 0.8221
Epoch 6/30 - Train Loss: 0.0064, Train Acc: 0.9985 | Test Loss: 0.9937, Test Acc: 0.8301
Epoch 7/30 - Train Loss: 0.0063, Train Acc: 0.9983 | Test Loss: 1.3272, Test Acc: 0.7853
Epoch 8/30 - Train Loss: 0.0109, Train Acc: 0.9960 | Test Loss: 1.5981, Test Acc: 0.7308
Epoch 9/30 - Train Loss: 0.0045, Train Acc: 0.9988 | Test Loss: 1.0676, Test Acc: 0.8317
Epoch 10/30 - Train Loss: 0.0028, Train Acc: 0.9990 | Test Loss: 1.4881, Test Acc: 0.7772
Epoch 11/30 - Train Loss: 0.0009, Train Acc: 1.0000 | Test Loss: 1.4355, Test Acc: 0.7788
Epoch 12/30 - Train

In [28]:
torch.save(model.state_dict(), "best_model.pth")
print("Model saved.")


Model saved.


In [ ]:
# Cell: GradCAM implementation for EfficientNet-B0

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []
        self._register_hooks()
        
    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        
        handle_forward = self.target_layer.register_forward_hook(forward_hook)
        handle_backward = self.target_layer.register_backward_hook(backward_hook)
        self.hook_handles.extend([handle_forward, handle_backward])
    
    def generate_cam(self, input_tensor, class_idx=None):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        score = output[0, class_idx]
        score.backward()
        
        # Global average pooling of gradients
        gradients = self.gradients  # shape: [1, C, H, W]
        activations = self.activations  # shape: [1, C, H, W]
        weights = torch.mean(gradients, dim=(2, 3), keepdim=True)  # shape: [1, C, 1, 1]
        cam = torch.sum(weights * activations, dim=1)  # shape: [1, H, W]
        cam = torch.relu(cam)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        cam = cam.cpu().numpy()[0]
        cam = cv2.resize(cam, (224, 224))
        return cam

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()


def show_gradcam(model, img_tensor, true_label, class_names):
    model.eval()
    # For EfficientNet-B0, target the last convolutional layer via 'conv_head'
    target_layer = model.conv_head
    grad_cam = GradCAM(model, target_layer)
    
    input_tensor = img_tensor.unsqueeze(0).to(device)
    cam = grad_cam.generate_cam(input_tensor)
    
    # Denormalize the image for visualization (using ImageNet normalization)
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    img_denorm = inv_normalize(img_tensor).cpu().numpy().transpose((1,2,0))
    img_denorm = np.clip(img_denorm, 0, 1)
    
    # Create a heatmap and overlay it on the original image
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlay = heatmap + np.float32(img_denorm)
    overlay = overlay / np.max(overlay)
    
    # Get the model's prediction
    output = model(input_tensor)
    _, pred = torch.max(output, 1)
    pred_label = class_names[pred.item()]
    true_label_str = class_names[true_label]
    
    # Plot side-by-side the original image and the GradCAM overlay
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img_denorm)
    axs[0].set_title(f"Original\nTrue: {true_label_str}")
    axs[0].axis('off')
    axs[1].imshow(overlay)
    axs[1].set_title(f"GradCAM Overlay\nPredicted: {pred_label}")
    axs[1].axis('off')
    plt.show()
    
    grad_cam.remove_hooks()

# Example usage:
# Ensure you have a test DataLoader named 'test_loader' and the class_names list (e.g., ['NORMAL', 'PNEUMONIA'])
inputs, labels = next(iter(test_loader))
idx = random.randint(0, inputs.size(0) - 1)
show_gradcam(model, inputs[idx], labels[idx].item(), class_names)
